---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
def degree_distribution(G):
    
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
    
    return histogram


def graph_identification():
    
    methods = []
    
    for G in P1_Graphs:
        
        degDis = degree_distribution(G)
        avgClu = nx.average_clustering(G)
        
        if len(degDis) > 10:
            methods.append('PA')
            
        elif avgClu < 0.1:
            methods.append('SW_H')
            
        else:
            methods.append('SW_L')
            
    return methods

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
def salary_predictions():
    from sklearn.ensemble import GradientBoostingClassifier
    
    df = pd.DataFrame()
    target = nx.get_node_attributes(G, 'ManagementSalary')
    
    degree = nx.degree_centrality(G)
    closeness = nx.closeness_centrality(G)
    betweenness = nx.betweenness_centrality(G)
    
    for node in target.keys():
        
        features = pd.DataFrame([[degree[node], 
                                  closeness[node],
                                  betweenness[node], 
                                  target[node]]], 
                                  index = [node])
        
        df = pd.concat([df, features], axis=0)
        
    train = df[~df[3].isnull()]
    test = df[df[3].isnull()]
    X_train = train[[0, 1, 2]].values
    X_test  = test[[0, 1, 2]].values
    y_train = train[3].values
    
    mGBC   = GradientBoostingClassifier().fit(X_train, y_train)
    yPred  = mGBC.predict_proba(X_test)[:,1] 
    
    salary = pd.Series(yPred, index = test.index)
    
    return salary

salary_predictions()

1       0.056806
2       0.987200
5       0.986621
8       0.083907
14      0.241456
18      0.591882
27      0.105213
30      0.280093
31      0.691004
34      0.019853
37      0.067486
40      0.063245
45      0.057983
54      0.281733
55      0.102085
60      0.131825
62      0.986621
65      0.987200
77      0.064108
79      0.162649
97      0.008753
101     0.008943
103     0.201927
108     0.224330
113     0.034396
122     0.023352
141     0.175558
142     0.986621
144     0.016478
145     0.044571
          ...   
913     0.007418
914     0.013940
915     0.008724
918     0.013283
923     0.006392
926     0.015300
931     0.012677
934     0.006392
939     0.006392
944     0.008724
945     0.006392
947     0.008724
950     0.125530
951     0.006392
953     0.009612
959     0.008724
962     0.006392
963     0.068232
968     0.008724
969     0.008724
974     0.008057
984     0.006392
987     0.008057
989     0.008724
991     0.008724
992     0.008724
994     0.008724
996     0.0087

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
def new_connections_predictions():  

    from sklearn.ensemble import GradientBoostingClassifier
    
    future_connections['pfAtt'] = [list(nx.preferential_attachment(G, [pair]))[0][2]
                                          for pair in future_connections.index]
    
    future_connections['comNei'] = [len(list(nx.common_neighbors(G, pair[0], pair[1]))) 
                                             for pair in future_connections.index]
    
    train = future_connections[~future_connections['Future Connection'].isnull()]
    test = future_connections[future_connections['Future Connection'].isnull()]
    X_train = train[['pfAtt','comNei']].values
    X_test  = test[['pfAtt','comNei']].values
    y_train = train['Future Connection'].values
   
    mGBC = GradientBoostingClassifier().fit(X_train, y_train)
    yPred = mGBC.predict_proba(X_test)[:,1]    
    futureCon = pd.Series(yPred, index = test.index)
    
    return futureCon

new_connections_predictions()

(107, 348)    0.031823
(542, 751)    0.012931
(20, 426)     0.543026
(50, 989)     0.013104
(942, 986)    0.013103
(324, 857)    0.013104
(13, 710)     0.148865
(19, 271)     0.141496
(319, 878)    0.013104
(659, 707)    0.012887
(49, 843)     0.013104
(208, 893)    0.012887
(377, 469)    0.012221
(405, 999)    0.022630
(129, 740)    0.017730
(292, 618)    0.017926
(239, 689)    0.013104
(359, 373)    0.012696
(53, 523)     0.032456
(276, 984)    0.013104
(202, 997)    0.013104
(604, 619)    0.119968
(270, 911)    0.013104
(261, 481)    0.069337
(200, 450)    0.825813
(213, 634)    0.012931
(644, 735)    0.088258
(346, 553)    0.012970
(521, 738)    0.012503
(422, 953)    0.017730
                ...   
(672, 848)    0.013104
(28, 127)     0.979748
(202, 661)    0.012998
(54, 195)     0.998514
(295, 864)    0.013104
(814, 936)    0.012931
(839, 874)    0.013103
(139, 843)    0.013104
(461, 544)    0.013183
(68, 487)     0.013183
(622, 932)    0.012887
(504, 936)    0.017232
(479, 528) 